In [3]:
from __future__ import print_function
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

pd.set_option('display.max_columns', None)


In [18]:
data = pd.read_csv('adult.dat',header=None, names= ['age','workclass','fnlwgt','education','edu-num','marital-status','occupation',
                                                    'relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                                                    'native-country','class'])
                   
                   
                   
                   
                   

In [14]:
np.shape(pd.get_dummies(data).values)

(32561, 110)

In [15]:

data_array = data.values


In [16]:
good_rows = np.invert(np.any(data_array=='?',1))
data = data_array[good_rows]

In [17]:
np.shape(data)

(32561, 15)

Lots of missing data, keep columns as Q? is that a category? Think about it but maybe
Could also skip those

Either way let's set up ANN using TF!

In [ ]:
# hyperparameters, adjust these like a knob
learning_rate = 0.1
num_steps = 500
batch_size = 500
display_step = 1

# Network Parameters, can play with these too, def should be larger based on this being a huge dataset. 
n_hidden_1 = 12 # 1st layer number of neurons
n_hidden_2 = 8 # 2nd layer number of neurons


num_input = # will know this number after I figure out how to deal with the dummies + missing data
num_classes = 2 # over or under 50k a year

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
# Store layers weight & biasm only 2 layers at the moment but seemingly not hard to fix
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with a specifid amoutn of neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    activation_1 = tf.nn.relu(layer_1)
    #transitions occur after applying an activation, ooooh
    layer_2 = tf.add(tf.matmul(activiation_1, weights['h2']), biases['b2'])
    activation_2 = tf.nn.sigmoid(layer_2)
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(activation_2, weights['out']) + biases['out']
    return out_layer

In [ ]:

# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))   #Loss function ignore softmax but crossentropy is a categorization loss func good for this prob
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,x_train,y_train)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test,
                                      Y: y_test}))
    
    
    
#Run through to make a NN, take note of input labels and classes and adjust names accordingly.